<a href="https://colab.research.google.com/github/NisargShah07/ObjectDetection/blob/main/ResNetDenseNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow
import tensorflow.keras as keras

In [ ]:
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

In [ ]:
model1=keras.applications.ResNet101V2(include_top=False,input_shape=(32,32,3))
flat1=keras.layers.Flatten()(model1.layers[-1].output)
model2=keras.applications.DenseNet201(include_top=False,input_shape=(32,32,3))
flat2=keras.layers.Flatten()(model2.layers[-1].output)

In [ ]:
for layer in model2.layers:
    layer._name=layer._name+'2'

In [ ]:
out=model1.layers[-1].output
def model_custom():
  # input_layer=keras.layers.Input(shape=(224,224,3))
  # model1=keras.applications.DenseNet201(include_top=False,input_shape=(32,32,3))
  # flat1=keras.layers.Flatten()(model1.layers[-1].output)
  concatLayer=keras.layers.Concatenate(axis=1)([flat1,flat2])
  layer1=keras.layers.Dense(units=256,activation='relu')(concatLayer)
  layer4=keras.layers.Dense(units=128,activation='relu')(layer1)
  layer7=keras.layers.Dense(units=10,activation='softmax')(layer4)
  model=keras.Model(inputs=[model1.inputs,model2.inputs],outputs=layer7)
  return model

In [ ]:
model=model_custom()
model.summary()

In [ ]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['acc'])

In [ ]:
model.fit([x_train,x_train],y_train,batch_size=64,epochs=30,validation_split=0.2)

In [ ]:
loss,acc=model.evaluate([x_test,x_test],y_test)
print(acc)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
def plot_ResNet_CAM(orig_image_path,pred_class ):
  
    im = image.load_img(orig_image_path, target_size=(224, 224))
    # plot image
    fig, ax = plt.subplots(nrows=1, ncols=2)
    fig.set_size_inches(10, 20)
    ax[0].imshow(im)
    ax[0].set_title('Original Image')
    ax[0].axis('off')
    
    ax[1].imshow(im, alpha=0.5)
    # ax[1].imshow(CAM, cmap='jet', alpha=0.5)
    ax[1].axis('off')
    ax[1].set_title(pred_class)
   
    # get the category name
    # with open(imagenet100_clsid_to_text_path) as imagenet_classes_file:
    #     imagenet_classes_dict = ast.literal_eval(imagenet_classes_file.read())
    # ax[1].set_title(imagenet_classes_dict[pred_class].split(',')[0])    
    plt.show()

In [ ]:
import cv2
from keras.preprocessing import image 
import matplotlib.pyplot as plt
img_path="/content/images.jpg"
img=cv2.imread(img_path)
img = cv2.resize(img,(32,32),3)
img=np.array(img)
img = np.expand_dims(img, axis=0)
pred_class=model.predict([img,img])
plot_ResNet_CAM(img_path, pred_class )